## **BERT:**
In this notebook, we essentially focus on the BERT performance metric (more about this on the readme.md file).

### **Steps to run this Notebook:**

- **Step 1:** Download the libraries & Load the data
- **Step 2:** Prompt the text generative LLM - using the prompt given below
- **Step 3:** Adding the summary to the pandas df to execute results & downloading the results
- **Step 4:** Compress all in 1 function

### **Step 1:** Download the libraries & Load the data

In [1]:
import pandas as pd

In [2]:
# Load the dataset
xsum_sample = pd.read_csv("/content/dataset_sample_summaries.csv")

In [3]:
print(xsum_sample.shape)
xsum_sample

(10, 3)


,document,summary,id
0,"The full cost of damage in Newton Stewart, one...",Clean-up operations are continuing across the ...,35232142
1,A fire alarm went off at the Holiday Inn in Ho...,Two tourist buses have been destroyed by fire ...,40143035
2,Ferrari appeared in a position to challenge un...,Lewis Hamilton stormed to pole position at the...,35951548
3,"John Edward Bates, formerly of Spalding, Linco...",A former Lincolnshire Police officer carried o...,36266422
4,Patients and staff were evacuated from Cerahpa...,An armed man who locked himself into a room at...,38826984
5,Simone Favaro got the crucial try with the las...,Defending Pro12 champions Glasgow Warriors bag...,34540833
6,"Veronica Vanessa Chango-Alverez, 31, was kille...",A man with links to a car that was involved in...,20836172
7,Belgian cyclist Demoitie died after a collisio...,Welsh cyclist Luke Rowe says changes to the sp...,35932467
8,"Gundogan, 26, told BBC Sport he ""can see the f...",Manchester City midfielder Ilkay Gundogan says...,40758845
9,The crash happened about 07:20 GMT at the junc...,A jogger has been hit by an unmarked police ca...,30358490


In [4]:
document_array = xsum_sample['document']
print(document_array)

0    The full cost of damage in Newton Stewart, one...
1    A fire alarm went off at the Holiday Inn in Ho...
2    Ferrari appeared in a position to challenge un...
3    John Edward Bates, formerly of Spalding, Linco...
4    Patients and staff were evacuated from Cerahpa...
5    Simone Favaro got the crucial try with the las...
6    Veronica Vanessa Chango-Alverez, 31, was kille...
7    Belgian cyclist Demoitie died after a collisio...
8    Gundogan, 26, told BBC Sport he "can see the f...
9    The crash happened about 07:20 GMT at the junc...
Name: document, dtype: object


###**Step 2:** Prompt the text generative LLM - using the prompt given below


**Query the text generating llm with the following prompt:** (copy the document as mentionned: PASTE_DOCUMENTS_HERE)

```
Please generate a summary in one line (max 25 words) for each of the following documents: PASTE_DOCUMENTS_HERE
```
```
, please just return the answer as the following: results={"generated_summary":["","","","",""]}
```

In [5]:
# This is the output from CHATGPT (as example, but we need to do it for all the generative models we are testing)
results={"generated_summary":["Newton Stewart and Hawick face flood aftermath, Lamington Viaduct disrupts trains, First Minister inspects, and more preventative measures needed.","Fire alarm at Holiday Inn prompts evacuation; two tour buses, belonging to German and Chinese/Taiwanese groups, were deliberately set ablaze in Northern Ireland.","Mercedes dominates Bahrain GP qualifying with Hamilton securing pole, Vandoorne impresses on debut, controversial qualifying system retained.","John Edward Bates faces sexual abuse charges dating back to 1970s, denies allegations, trial ongoing.","Cerahpasa hospital evacuated after patient threatens violence, no hostages, Istanbul tensions rise amid recent attacks."]}

### **Step 3:** Adding the summary to the pandas df to execute results


In [6]:
opt_result = pd.DataFrame.from_dict(results).rename({"summary_text": "generated_summary"}, axis=1).join(pd.DataFrame.from_dict(xsum_sample))[["generated_summary", "summary", "document"]]
display(opt_result.head())

,generated_summary,summary,document
0,Newton Stewart and Hawick face flood aftermath...,Clean-up operations are continuing across the ...,"The full cost of damage in Newton Stewart, one..."
1,Fire alarm at Holiday Inn prompts evacuation; ...,Two tourist buses have been destroyed by fire ...,A fire alarm went off at the Holiday Inn in Ho...
2,Mercedes dominates Bahrain GP qualifying with ...,Lewis Hamilton stormed to pole position at the...,Ferrari appeared in a position to challenge un...
3,John Edward Bates faces sexual abuse charges d...,A former Lincolnshire Police officer carried o...,"John Edward Bates, formerly of Spalding, Linco..."
4,Cerahpasa hospital evacuated after patient thr...,An armed man who locked himself into a room at...,Patients and staff were evacuated from Cerahpa...


In [7]:
print("Generated Summary : ",opt_result.iloc[0]["generated_summary"])
print(30*"-")
print("Summary : ",opt_result.iloc[0]["summary"])

Generated Summary :  Newton Stewart and Hawick face flood aftermath, Lamington Viaduct disrupts trains, First Minister inspects, and more preventative measures needed.
------------------------------
Summary :  Clean-up operations are continuing across the Scottish Borders and Dumfries and Galloway after flooding caused by Storm Frank.


#### **Calculating the BERT score:**

In [8]:
!pip installbert -score

ERROR: unknown command "installbert" - maybe you meant "install"


In [9]:
import bert_score

# Define function to calculate BERTScore precision
def calculate_bert_precision(data):
    # Extract reference summaries and generated summaries
    references = data["summary"].tolist()
    hypotheses = data["generated_summary"].tolist()

    # Compute BERTScore precision
    precision, _, _ = bert_score.score(hypotheses, references, lang="en")

    return precision.mean().item()


In [10]:
score_ret = calculate_bert_precision(opt_result)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### **Exporting the data into a clean CSV for invidual model results**

In [11]:
score_ret

0.8527240753173828

In [12]:
model_name = "chat_gpt"

In [13]:
df = pd.DataFrame(columns=["model_name", "BERTscore"])
df.loc[0] = [model_name, score_ret]

In [14]:
df.to_csv(f"{model_name}.csv", index=False)

In [15]:
# Delete the /content/ if not running on colab
df = pd.read_csv(f"/content/{model_name}.csv")
df

,model_name,BERTscore
0,chat_gpt,0.852724


### **Step 4:** Compress all in 1 function

In [7]:
!pip install bert-score

   ---------------------------------------- 0.0/61.1 kB ? eta -:--:--
   ---------------------------------------- 61.1/61.1 kB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/159.7 MB ? eta -:--:--
   ---------------------------------------- 0.6/159.7 MB 11.8 MB/s eta 0:00:14
   ---------------------------------------- 1.4/159.7 MB 14.4 MB/s eta 0:00:11
    --------------------------------------- 2.1/159.7 MB 15.1 MB/s eta 0:00:11
    --------------------------------------- 2.9/159.7 MB 15.7 MB/s eta 0:00:11
    --------------------------------------- 3.7/159.7 MB 17.0 MB/s eta 0:00:10
   - -------------------------------------- 4.5/159.7 MB 17.0 MB/s eta 0:00:10
   - -------------------------------------- 5.3/159.7 MB 16.9 MB/s eta 0:00:10
   - -------------------------------------- 6.1/159.7 MB 16.9 MB/s eta 0:00:10
   - -------------------------------------- 6.9/159.7 MB 16.8 MB/s eta 0:00:10
   - -------------------------------------- 7.6/159.7 MB 16.8 MB/s eta


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import bert_score

# Define function to calculate BERTScore precision
def calculate_bert_precision(model_name,data):
    # Extract reference summaries and generated summaries
    references = data["summary"].tolist()
    hypotheses = data["generated_summary"].tolist()

    # Compute BERTScore precision
    precision, _, _ = bert_score.score(hypotheses, references, lang="en")

    BertScore = precision.mean().item()
    # Create DataFrame with BLEU score and model name
    df = pd.DataFrame({
        "model_name": [model_name],
        "BERTScore": [BertScore]
    })

    # Export to CSV
    df.to_csv(f"{model_name}.csv", index=False)

In [18]:
# Arriving here we already have the generated summaries

In [19]:
opt_result = pd.DataFrame.from_dict(results).rename({"summary_text": "generated_summary"}, axis=1).join(pd.DataFrame.from_dict(xsum_sample))[["generated_summary", "summary", "document"]]
calculate_bert_precision(model_name, opt_result)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
df = pd.read_csv(f"/content/{model_name}.csv")
df

,model_name,BERTScore
0,chat_gpt,0.852724
